# Spark Apache (семинары)

## Урок 4. Изучение Spark Structure Streaming

### Задача:

- Условие: используйте источник rate, напишите код, который создаст дополнительный столбец, который будет выводить сумму только нечётных чисел.

In [1]:
import init_spark_env

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr